# Chapter 5 트리 알고리즘

학습목표 :  
- 성능이 좋고 이해하기 시운 트리 알고리즘에 대해 배움
- 알고리즘의 성능을 최대화하기 위한 하이퍼파라미터 튜닝을 실습
- 여러 트리를 합쳐 일반화 성능을 높일 수 있는 앙상블 모델을 배움

## 5-2 교차 검증과 그리드 서치

- 검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 배움. 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터를 찾음

In [2]:
import pandas as pd
wine=pd.read_csv("https://bit.ly/wine_csv_data")

In [3]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42) #train_set에서 한번 더 나눠서 검증 세트를 만드는 과정
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [6]:
from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.00912166, 0.00983191, 0.01037788, 0.00742531, 0.00704479]), 'score_time': array([0.0019803 , 0.00122643, 0.00098157, 0.00091791, 0.00128222]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np
print(np.mean(scores['test_score'])) #검증 폴드의 점수

0.855300214703487


In [8]:
from sklearn.model_selection import StratifiedKFold
scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42) #n_splits 매개변수는 몇 폴드 교차 검증을 할지 정함
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


In [10]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [11]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [12]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt=gs.best_estimator_ #grid search로 찾은 최적의 매개변수
print(dt.score(train_input,train_target))

0.9615162593804117


In [17]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수를 지정
2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. 이 조합은 그리드 서치 객체에 저장
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련. 이 모델도 그리드 서치 객체에 저장

In [20]:
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}

In [21]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [22]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [23]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

- random search (랜덤 서치) : 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어렵거나, 너무 많은 매개변수 조건이 있어 그리드 서치 수행시간이 오래 걸릴 때 사용

- scipy(싸이파이) : 파이썬의 과학 라이브러리로 수치 계산 전용 라이브러리

In [24]:
from scipy.stats import uniform,randint

In [26]:
rgen=randint(0,10)
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110, 118,  85, 103, 100,  90,  85, 110,  97, 102]))

In [27]:
ugen=uniform(0,1)
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25),}

In [28]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f5795fc3fa0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f5795fc14b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f5795fc07f0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f5795fc0250>},
                   random_state=42)

In [29]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [31]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86


### 마무리
- `검증 세트` : 하이퍼 파라미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어낸 데이터 세트
- `교차 검증` : 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트를 역할을 하고 나머지 폴드에서는 모델을 훈련. 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법
- `그리드 서치` : 하이퍼파라미터 탐색을 자동화해 주는 도구. 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택 후, 이 매개변수 조합으로 최종 모델을 훈련
- `랜덤 서치` : 연속된 매개변수 값을 탐색할 때 유용. 탐색 값을 샘플링 할 수 있는 확률 분포 객체를 전달
